In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install -q langchain bitsandbytes accelerate ping3 langchain-community langchain-core --use-deprecated=legacy-resolver

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.6/983.6 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 57.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.4/362.4 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 47.2 MB/s eta 0:00:00


In [ ]:
import gc, inspect, json, re
import xml.etree.ElementTree as ET
from functools import partial
from typing import get_type_hints

import transformers
import torch

from langchain.chains.openai_functions import convert_to_openai_function
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
from langchain.pydantic_v1 import BaseModel, Field, validator

import torch
from transformers import BitsAndBytesConfig
from langchain import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline


In [ ]:
from langchain.pydantic_v1 import BaseModel, Field, validator

In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

In [ ]:
!pip install scapy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scapy: filename=scapy-2.5.0-py2.py3-none-any.whl size=1444327 sha256=39fe82ba23716f6507d1096b1a894257c64e0150f59c196cabee552b030cfc1e
  Stored in directory: /root/.cache/pip/wheels/82/b7/03/8344d8cf6695624746311bc0d389e9d05535ca83c35f90241d
Successfully built scapy


In [ ]:
import ping3
import time
import requests
import psutil
import time
import subprocess
from scapy.all import *
import os
import numpy as np

In [ ]:
model_name = "teknium/OpenHermes-2.5-Mistral-7B"

In [ ]:
def load_model(model_name: str):
    tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

    with torch.device("cuda:0"):
        model = transformers.AutoModelForCausalLM.from_pretrained(model_name, device_map="auto",quantization_config=quantization_config,).eval()

    return tokenizer, model

In [ ]:
tokenizer, model = load_model(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/101 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

In [ ]:
def delete_model(*args):
    for var in args:
        if var in globals():
            del globals()[var]

    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
class NetworkDevice(BaseModel):
  '''to ping network devices and check their availability.'''
  check_network: str = Field(description="question to check the availability of the network devices by pinging.")
  response: str = Field(description="response to ping request to check the availability")

  @validator("check_network")
  def ping_device(ip_address):
    '''function to ping network devices and check their availability.'''
    response_time = ping3.ping(ip_address)
    if response_time:
      return f"Device {ip_address} is up. Response time: {response_time} m"
    else:
        return f'Network device {ip_address} is down.'

class WebService(BaseModel):
  '''to check the status of web services.'''
  check_web: str = Field(description="question to check the status of web services.")
  response: str = Field(description="response to check the status of web services.")

  @validator("check_web")
  def check_service(url):
    '''function to check the status of web services.'''
    try:
      response = requests.get(url)
      if response.status_code == 200:
        return print(f'Service {url} is up. Response time: {response.elapsed.total_seconds()} seconds')
      else:
        return f'Web Service {url} is down.'
    except requests.exceptions.RequestException:
      return f'Web Service {url} is down.'

class BandwithUsage(BaseModel):
  '''to check amount of bandwith usage.'''
  check_bandwith_usage: str = Field(description="question to check amount of bandwith usage.")
  response: str = Field(description="response to check amount of bandwith usage.")

  @validator("check_bandwith_usage")
  def get_bandwidth(interface):
    net_io = psutil.net_io_counters(pernic=True)[interface]
    return f"Sent: {net_io.bytes_sent} bytes, Received: {net_io.bytes_recv} bytes"

class PacketLoss(BaseModel):
  '''to check amount of packet loss.'''
  check_packet_loss: str = Field(description="question to check  packet loss.")
  response: str = Field(description="response to check packet loss.")

  @validator("check_packet_loss")
  def check_packetloss(host):
    response = subprocess.run(['ping', '-c', '4', host], stdout=subprocess.PIPE)
    output = response.stdout.decode()
    if "0% packet loss" in output:
      return f"{host} is reachable with no packet loss."
    else:
      return f"{host} is unreachable or has packet loss."

class Retransmission(BaseModel):
  '''to monitor retransmission to analyze packet flows..'''
  check_retransmission: str = Field(description="question to monitor retransmission to analyze packet flows..")
  response: str = Field(description="response to monitor retransmission to analyze packet flows.")

  @validator("check_retransmission")
  def check_re_transmission(interface):
    def packet_callback(packet):
        if 'TCP' in packet and packet['TCP'].flags == 'R':
            return f'Retransmission detected: {packet.summary()}'
    sniff(iface=interface, prn=packet_callback)

"""class Throughput(BaseModel):
  '''to calculate throughput by measuring data sent over time.'''
  calculatethroughput: str = Field(description="question to calculate throughput by measuring data sent over time.")
  response: str = Field(description="response to calculate throughput by measuring data sent over time.")

  @validator("calculatethroughput")
  def calculate_throughput():
    start_time = time.time()
    start_bytes = psutil.net_io_counters().bytes_sent + psutil.net_io_counters().bytes_recv

    time.sleep(1)  # Measure for 1 second

    end_bytes = psutil.net_io_counters().bytes_sent + psutil.net_io_counters().bytes_recv
    throughput = end_bytes - start_bytes
    return f'Throughput: {throughput} bytes/second' """

class Latency(BaseModel):
  '''to measure round-trip time (RTT) or latency of the host'''
  calculatelatency: str = Field(description="question to measure round-trip time (RTT) or latency of the host")
  response: str = Field(description="response to measure round-trip time (RTT) or latency of the host")

  @validator("calculatelatency")
  def calculate_latency(host):
    response = subprocess.run(['ping', '-c', '1', host], stdout=subprocess.PIPE)
    output = response.stdout.decode()
    latency = output.split('time=')[-1].split(' ms')[0]
    return f'Latency to {host}: {latency} ms'

class Jitter(BaseModel):
  '''to Calculate jitter by measuring the variation in latency over multiple pings'''
  calculatejitter: str = Field(description="to Calculate jitter by measuring the variation in latency over multiple pings")
  response: str = Field(description="response to Calculate jitter by measuring the variation in latency over multiple pings")

  @validator("calculatejitter")
  def calculate_jitter(host):
    latencies = []
    for _ in range(5):
        latencies.append(calculate_latency(host))
        time.sleep(1)
    jitter = np.std(latencies)
    return f'Jitter to {host}: {jitter} ms'

In [ ]:
def check_service(url: str) -> str:
    '''Function to check the status of web services.'''
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return f'Service {url} is up. Response time: {response.elapsed.total_seconds()} seconds'
        else:
            return f'Web Service {url} is down.'
    except requests.exceptions.RequestException:
        return f'Web Service {url} is down.'

def ping_device(ip_address):
    '''function to ping network devices and check their availability.'''
    response_time = ping3.ping(ip_address)
    if response_time:
      return f"Device {ip_address} is up. Response time: {response_time} m"
    else:
        return f'Network device {ip_address} is down.'

def get_bandwidth(interface):
  net_io = psutil.net_io_counters(pernic=True)[interface]
  return f"Sent: {net_io.bytes_sent} bytes, Received: {net_io.bytes_recv} bytes"

def check_packetloss(host):
  response = subprocess.run(['ping', '-c', '4', host], stdout=subprocess.PIPE,shell=True)
  output = response.stdout.decode()
  if "0% packet loss" in output:
    return f"{host} is reachable with no packet loss."
  else:
    return f"{host} is unreachable or has packet loss."

def check_re_transmission(interface):
  def packet_callback(packet):
    if 'TCP' in packet and packet['TCP'].flags == 'R':
      return f'Retransmission detected: {packet.summary()}'
  sniff(iface=interface, prn=packet_callback)

def calculate_throughput():
  start_time = time.time()
  start_bytes = psutil.net_io_counters().bytes_sent + psutil.net_io_counters().bytes_recv
  time.sleep(1)  # Measure for 1 second
  end_bytes = psutil.net_io_counters().bytes_sent + psutil.net_io_counters().bytes_recv
  throughput = end_bytes - start_bytes
  return f'Throughput: {throughput} bytes/second'

def calculate_latency(host):
  response = subprocess.run(['ping', '-c', '1', host], stdout=subprocess.PIPE)
  output = response.stdout.decode()
  latency = output.split('time=')[-1].split(' ms')[0]
  return f'Latency to {host}: {latency} ms'

def calculate_jittter(host):
    latencies = []
    for _ in range(5):
        latencies.append(calculate_latency(host))
        time.sleep(1)
    jitter = np.std(latencies)
    return f'Jitter to {host}: {jitter} ms'

In [ ]:
convert_pydantic_to_openai_function(NetworkDevice)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The function `convert_pydantic_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 0.3.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  warn_deprecated(


{'name': 'NetworkDevice',
 'description': 'to ping network devices and check their availability.',
 'parameters': {'type': 'object',
  'properties': {'check_network': {'description': 'question to check the availability of the network devices by pinging.',
    'type': 'string'},
   'response': {'description': 'response to ping request to check the availability',
    'type': 'string'}},
  'required': ['check_network', 'response']}}

In [ ]:
def extract_function_calls(completion):
    completion = completion.strip()
    completion = completion.replace("<|im_end|>","")
    res = eval(f"""{completion}""")
    if res['name']=="NetworkDevice":
      return ping_device(res["arguments"]["check_network"])

    elif res['name']=="WebService":
      return check_service(res["arguments"]["check_web"])

    elif res['name']=="BandwithUsage":
      return check_service(res["arguments"]["check_bandwith_usage"])

    elif res['name']=="PacketLoss":
      return check_packetloss(res["arguments"]["check_packet_loss"])

    elif res['name']=="Retransmission":
      return check_re_transmission(res["arguments"]["check_retransmission"])

    elif res['name']=="Latency":
      return calculate_latency(res["arguments"]["calculatelatency"])
    else:
      return calculate_jitter(res["arguments"]["calculatejitter"])



In [ ]:

def generate_hermes(prompt, model, tokenizer, generation_config_overrides={}):
    fn = """{"name": "function_name", "arguments": {"arg_1": "value_1", "arg_2": value_2, ...}}"""
    prompt = f"""<|im_start|>system
    You are a helpful assistant with access to the following functions:
    {convert_pydantic_to_openai_function(NetworkDevice)}

    {convert_pydantic_to_openai_function(WebService)}

    {convert_pydantic_to_openai_function(BandwithUsage)}

    {convert_pydantic_to_openai_function(PacketLoss)}

    {convert_pydantic_to_openai_function(Retransmission)}

    {convert_pydantic_to_openai_function(Latency)}

    {convert_pydantic_to_openai_function(Jitter)}

    To use these functions respond with:

     {fn}
     {fn}
    ...


  Edge cases you must handle:
  - If there are no functions that match the user request, you will respond politely that you cannot help.<|im_end|>
  <|im_start|>user
  {prompt}<|im_end|>
  <|im_start|>assistant"""

    generation_config = model.generation_config
    generation_config.update(
        **{
            **{
                "use_cache": True,
                "do_sample": True,
                "temperature": 0.2,
                "top_p": 1.0,
                "top_k": 0,
                "max_new_tokens": 512,
                "eos_token_id": tokenizer.eos_token_id,
                "pad_token_id": tokenizer.eos_token_id,
            },
            **generation_config_overrides,
        }
    )

    model = model.eval()
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    n_tokens = inputs.input_ids.numel()

    with torch.inference_mode():
        generated_tokens = model.generate(**inputs, generation_config=generation_config)

    return tokenizer.decode(
        generated_tokens.squeeze()[n_tokens:], skip_special_tokens=False
    )


In [ ]:
%%time
generation_func = partial(generate_hermes, model=model, tokenizer=tokenizer)

CPU times: user 7 µs, sys: 0 ns, total: 7 µs
Wall time: 10.3 µs


In [ ]:
prompts = [
    "check whether the network device 192.168.1.1 is available.",
    "check whether the web service https://google.com is available.",
    "check amount of bandwith usage of eth0",
    "check amount of packet loss for the host = 8.8.8.8",
  ]
'''
    "calculate the latency of host 8.8.8.8 ",
    "calculate jitter of host 8.8.8.8"
'''

'\n    "calculate the latency of host 8.8.8.8 ",\n    "calculate jitter of host 8.8.8.8"\n'

In [ ]:
for prompt in prompts:
  print(prompt)
  completion = generation_func(prompt)
  print("Result from LLM:", completion)
  print()
  functions = extract_function_calls(completion)
  if functions:
    print(functions)
  else:
    print(completion.strip())
  print("="*100,"\n")

check whether the network device 192.168.1.1 is available.
Result from LLM: 
{"name": "NetworkDevice", "arguments": {"check_network": "check whether the network device 192.168.1.1 is available."}}<|im_end|>

Network device check whether the network device 192.168.1.1 is available. is down.

check whether the web service https://google.com is available.
Result from LLM: 
{"name": "WebService", "arguments": {"check_web": "https://google.com"}}<|im_end|>

Service https://google.com is up. Response time: 0.090555 seconds

check amount of bandwith usage of eth0
Result from LLM: 
{"name": "BandwithUsage", "arguments": {"check_bandwith_usage": "check amount of bandwith usage of eth0"}}<|im_end|>

Web Service check amount of bandwith usage of eth0 is down.

check amount of packet loss for the host = 8.8.8.8
Result from LLM: 
{"name": "PacketLoss", "arguments": {"check_packet_loss": "8.8.8.8"}}<|im_end|>

8.8.8.8 is unreachable or has packet loss.

